In [ ]:
!py -3 -m pip install pandas
!py -3 -m pip install teradatasql
!py -3 -m pip install teradatasqlalchemy

In [ ]:
import pandas as pd
# import teradatasql
import sqlalchemy
import os

In [ ]:
user_name = 'KHU9683'
pword = 'Mer82@dme'
host_name='EDWPROD.DW.MEDCITY.NET'

In [ ]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=LDAP')

In [ ]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\compare_tables.csv"
output_folder = "OutputFiles\\Validation_Scripts\\"
source_system = 'PBS'
mirrored_database_project = 'hca-hin-dev-cur-parallon'
mirrored_last_update_date = '2024-01-23'
tested_database_project = 'hca-hin-dev-cur-parallon'
# tested_dataset_copy_used = True
tested_last_update_date = '2024-01-23'
if mirrored_database_project == tested_database_project:
    tested_dataset_copy_used = True
else:
    tested_dataset_copy_used = False

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [ ]:
#get the DDLs for the tables from teradata 
def gen_validation_scripts():
    if not os.path.exists(output_folder + source_system):
        os.mkdir(output_folder + source_system)
# with teradatasql.connect(host=host_name, user=user_name, password=pword, logmech="LDAP") as connect:
    df = pd.read_csv(input_path, index_col=None)        
    
    exclude_column_list = ['dw_last_update_date_time', 'dw_last_update_date_tim',
    'sk_generated_date_time', 'dw_last_update_time',
    'valid_from_date', 'valid_to_date']
    for index, row in df.iterrows():
        try:
            database_name = str(row['Database']).strip().lower()
            table_name = str(row['Table']).strip().lower()
            valid_to_date_present = False

            select_list = []

            if database_name == "edwhr_staging":
                database_param = "{{ params.param_hr_stage_dataset_name }}"

            select_list.append("SELECT ")

            query = "Select LOWER(columnname) AS columnname, UPPER(columntype) AS columntype "\
                "From dbc.columnsV WHERE UPPER(DatabaseName) = '{}' AND UPPER(TableName) = '{}'"\
                ";".format(database_name, table_name)
            results_df = pd.read_sql(query, td_engine)

            valid_to_date_present = 'valid_to_date' in results_df['columnname'].unique()

            validate_column_list = [i for i in results_df['columnname'] if i not in exclude_column_list]

            if valid_to_date_present:
                update_type_list = ['insert','update']
            else:
                update_type_list = ['update']

            for row_no, record in enumerate(validate_column_list):
                select_list.append(chr(9) + record + ("," if((row_no + 1 ) < len(results_df)) else ""))
            select_list.append("FROM")
            
            for update_type in update_type_list:
                valid_to_date_check = ""
                if valid_to_date_present:
                    if update_type == 'insert':
                        valid_to_date_check = "AND DATE(VALID_TO_DATE) = '9999-12-31'"
                    else:
                        valid_to_date_check = "AND DATE(VALID_TO_DATE) <> '9999-12-31'"
                for comparison_type in ['QA_Minus_Prod','Prod_Minus_QA','Prod_Match_QA']:
                    compare_query = []
                    compare_query.extend(select_list)
                    if comparison_type == 'QA_Minus_Prod':
                        compare_query.append("{}.{}.{}"\
                            "".format(tested_database_project,
                            database_name+("_copy" if tested_dataset_copy_used else ""),
                            table_name))
                        compare_query.append("Where DATE(dw_last_update_date_time) = '{}'"\
                            "".format(tested_last_update_date))
                        if valid_to_date_present:
                            compare_query.append(valid_to_date_check)
                        compare_query.append("EXCEPT DISTINCT")
                        compare_query.extend(select_list)
                        compare_query.append("{}.{}.{}"\
                            "".format(mirrored_database_project, database_name + "_base_views", table_name))
                        compare_query.append("Where DATE(dw_last_update_date_time) = '{}'"\
                            "".format(mirrored_last_update_date))
                        if valid_to_date_present:
                            compare_query.append(valid_to_date_check)
                    elif comparison_type == 'Prod_Minus_QA':
                        compare_query.append("{}.{}.{}"\
                            "".format(mirrored_database_project, database_name + "_base_views", table_name))
                        compare_query.append("Where DATE(dw_last_update_date_time) = '{}'"\
                            "".format(mirrored_last_update_date))
                        if valid_to_date_present:
                            compare_query.append(valid_to_date_check)
                        compare_query.append("EXCEPT DISTINCT")
                        compare_query.extend(select_list)
                        compare_query.append("{}.{}.{}"\
                            "".format(tested_database_project,
                            database_name+("_copy" if tested_dataset_copy_used else ""),
                            table_name))
                        compare_query.append("Where DATE(dw_last_update_date_time) = '{}'"\
                            "".format(tested_last_update_date))
                        if valid_to_date_present:
                            compare_query.append(valid_to_date_check)
                    else:
                        compare_query.append("{}.{}.{}"\
                            "".format(mirrored_database_project, database_name + "_base_views", table_name))
                        compare_query.append("Where DATE(dw_last_update_date_time) = '{}'"\
                            "".format(mirrored_last_update_date))
                        if valid_to_date_present:
                            compare_query.append(valid_to_date_check)
                        compare_query.append("INTERSECT DISTINCT")
                        compare_query.extend(select_list)
                        compare_query.append("{}.{}.{}"\
                            "".format(tested_database_project,
                            database_name+("_copy" if tested_dataset_copy_used else ""),
                            table_name))
                        compare_query.append("Where DATE(dw_last_update_date_time) = '{}'"\
                            "".format(tested_last_update_date))
                        if valid_to_date_present:
                            compare_query.append(valid_to_date_check)
                    output_path = output_folder + source_system + ("\{}_{}_{}.sql".format(table_name, comparison_type, update_type))
                    write_file_local(output_path, compare_query)

        except Exception as e1:
            print(e1)
            print("Database : {}, Table : {}".format(database_name,table_name))
            pass

    print("Tables for Record Counts : ", len(df))

In [ ]:
print("Begin of Processing")

gen_validation_scripts()

print("End of Processing")